The Turán number of $C_5^-$
==============================

This notebook contains calculations for the Turán number of $C_5^-$

As the blowup of $K_4^-$ contains $C_5^-$, we can additionally assume 
that we work in $K_4^-$-free structures. 

1. The first cell sets up the combinatorial theory of $C_5^-$ and 
$K_4^-$-free 3-graphs (called TGp). In addition, it sets up the 
combinatorial theory on the same 3-graphs with vertices colored 
from 3 possible colors (called CTGp)

2. The second cell performs the basic calculation of upper bounding
edges in the theory. It gives the 1/4 + 1/1000 (in fact 0.250728)
upper bound.

3. That upper bound is used in the following cell. It lower bounds
k221-2*k311 and k222/k111 at the optimum.

4. There is a quick sanity check below that, to check the coefficients
of k221, k311, f222, k222

5. The last cell works in the colored theory, and shows that the bad
edges have number smaller than the missing good edges. It uses the upper
bound about k222/k111.

In [2]:
###
### This cell is just to set up the theory.
### In practice it is not needed, as the calculations (multiplication table and generated structures)
### are already done and saved. But it is here for completeness (and for re-runs from scratch)
###

from sage.algebras.flag_algebras import *

# These are helper functions, to deal with classical exclusion (not just induced)
def check_containment(smalls, larges):
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
TG.exclude()

# C5 minus
C5m = TG(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])

# flags of size 5
fl5 = TG.generate_flags(5)

# boolean vector indicating each element in fl5 if it has C5m
gs = check_containment([C5m], fl5)

# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])

# set the excluded structures. k4, k4m and all in fl5 containing C5m
exls = [k4, k4m] + [xx for ii, xx in enumerate(fl5) if not gs[ii]]
TG.exclude(exls)

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    ftype_union = [jj for ff in ftype_points for jj in ff]
    partt = list(ftype_points) + \
            [[ii for ii in range(n) if ii not in ftype_union]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# def _identify_hypergraph(n, ftype_points, edges):
#     g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
#               format='vertices_and_edges')
#     partt = [[ii] for ii in ftype_points] + \
#             [[ii for ii in range(n) if ii not in ftype_points]] + \
#             [list(range(n,n+len(edges)))]
#     blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
#     return (n, len(ftype_points), blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoC5m", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 9 55 1127
print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))




# This is code to create colored theories
# This is a default code for all color partition
def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    ftype_union = [jj for ff in ftype_points for jj in ff]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = list(ftype_points) + \
              [[ii for ii in range(n) if ii not in ftype_union]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))
    
    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
#     is_graph = (k==2)
#     color_number = sum(len(xx) for xx in order_partition)
#     edges = kwargs["edges"]
#     Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
#     g_parts = [[ii] for ii in ftype_points] + \
#               [[ii for ii in range(n) if ii not in ftype_points]]
#     ppadd = 0 if is_graph else len(edges)
#     g_verts = list(range(n+ppadd+color_number))
#     g_parts.append(list(range(n, n+ppadd)))

#     g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
#     if is_graph:
#         g_edges = list(edges)
#         for ii in range(color_number):
#             g_edges += [(xx, n+ii) for xx in Cs[ii]]
#     else:
#         g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
#         for ii in range(color_number):
#             g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
#     g = Graph([g_verts, g_edges], format='vertices_and_edges')
#     blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
#     return (n, len(ftype_points), blocks)

# This is also a default code for all color partition
def _generateCT(base_theory, k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    BT = base_theory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

# To make the default codes work for this specific case:
# The generator:
# Colors the elements of TGp (3-graphs without C5- and K4-), works on 3-uniform structures
# and the colors 0, 1, 2 are interchangeable (otherwise it would say [[0], [1], [2]]
def generate_colored(n):
    return _generateCT(TGp, 3, [[0, 1, 2]], n)

# Same for the identifier. Colors are interchangeable.
def identify_colored(n, ftype_points, edges, C0, C1, C2):
    return _identifyCT(3, [[0, 1, 2]], n, ftype_points, edges=edges, C0=C0, C1=C1, C2=C2)

# CTGp is the colored variant of TGp (NoC5m)
CTGp = CombinatorialTheory("ColoredNoC5m", generate_colored, identify_colored, edges=3, C0=1, C1=1, C2=1)

# sanity check, the number of flags with size 4, 5, 6
# should be 18 132 2840
print(len(CTGp.generate_flags(4)), len(CTGp.generate_flags(5)), len(CTGp.generate_flags(6)))

9 55 1127
18 132 2840


In [10]:
###
### This is where the actual calculation starts.
### This part just gives a standard upper bound on the number of edges
###

# degree equality 
p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]

pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])

standard_assums = [degree_difference, -degree_difference, pointed_edge-1/4]


#run the optimizer
max_edge = TGp.optimize_problem(TGp(3, edges=[[0, 1, 2]]), 7, maximize=True, positives=standard_assums)

#should be around  0.25072863788449695 < 1/4 + 1/1000
print("\n\n", max_edge, "\n\n")

TypeError: object of type 'int' has no len()

In [18]:
###
### This code block deals with the uncolored part of the calculation (minimizing k221-2*k311 and k222/k111)
###

p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]
pointed_edge = TGp(3, edges=[[0, 1, 2]], ftype=[0])


import itertools
# base edges for f222 (0, 1, 2) is the center and (3, 4, 5) is the good edge
be = [[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]
# additional edges for f222. These are the edges we can include while still not violating the no C5m condition.
ae = [[0, 4, 5], [1, 3, 5], [2, 3, 4]]

# f221 and f311 just a sum of two elements, so can write it explicitely. They are typed, so no automorphism consideration is needed
f221 = TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [0, 2, 4]]) + TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [0, 2, 4], [2, 3, 4]])
f311 = TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [1, 2, 4]]) + TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [1, 2, 4], [0, 3, 4]])

# f222, it is a sum of 8 flags, so simply use this iterator to loop through all subsets of the additional edges
f222 = TGp(6, ftype=[0, 1, 2], edges=be)
for ii in [1, 2, 3]:
    for xx in itertools.combinations(ae, int(ii)):
        f222 = f222 + TGp(6, ftype=[0, 1, 2], edges=be+list(xx))

# these are the projected values. Here the automorphism groups are taken care of.
k221 = f221.project()
k311 = f311.project()
k222 = f222.project()

#standard positivity constraints.

# edge density bounds, the edge <= 1/4 + 1/1000 is used here from the previous cell
edge_lower = TGp(3, edges=[[0, 1, 2]]) - 1/4
edge_upper = 1/4 + 1/1000 - TGp(3, edges=[[0, 1, 2]])

# alpha for some of the calculations
alpha = 1-(1/100)

# the list of positivity assumptions
uncolored_assums = [edge_lower, edge_upper, degree_difference, -degree_difference]

# This is for minimizing f222, which is a flag, so technically it is the quotient k222 / k111
min_f222 = optimize_problem_old(TGp, f222, 7, maximize=True, positives=uncolored_assums)

# This is for minimizing k221 - (2*alpha)*k311
# On the optimal construction k221 = 2*k311, so alpha guarantees this is slightly positive
#min_k2m3 = TGp.optimize_problem(k221 - (2*alpha)*k311, 7, maximize=False, positives=uncolored_assums)

#they should be around 0.19835273 and 0.00032040802
#sometimes the print statements get tangled in the output of the CSDP. But the variables are set,
#they can be checked in a different cell
#print("\n\n", min_f222, "\n\n", min_k2m3, "\n\n")

Ftypes constructed in 68.54s
Block sizes done in 0.09s
Block sizes are [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28, -1127, -204]
Calculating product matrices for 12 ftypes and 1127 structures
Ftype on 5 points with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] is complete: : 12it [05:11, 25.93s/it]
Table calculation done in 311.22s
Target and constraint calculation done in 1.38s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 0.19835273299570932
Iter:  1 Ap: 1.73e-01 Pobj: -8.3464635e+01 Ad: 6.51e-02 Dobj: -5.4249432e-01 
Iter:  2 Ap: 8.03e-01 Pobj: -4.2125702e+02 Ad: 2.77e-01 Dobj: -2.2768908e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -5.7091428e+02 Ad: 8.39e-01 Dobj: -5.0876347e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -5.8095433e+02 Ad: 8.78e-01 Dobj: -1.1984328e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -5.7888658e+02 Ad: 8.51e-01 Dobj: -6.3583532e-02 
Iter:  6 Ap: 1.00e+00 Pobj: -7.5226314e+02 Ad: 4.81e-01 Dobj: -5.8745623e-02 
Iter:  7 Ap: 2.40e-0

In [ ]:
# change objective functions such that only bipartite is optimal.
# number of 1-1-1 edges
# difference between parts

# or just do the extended kernel thing from multiple constructions

In [4]:
# to check the coefficients. These sanity checks are easy to do in general
print("k221 is: ", k221, "\n\nk311 is: ", k311, "\n\nf222 is: ", f222, "\n\nk222 is: ", k222)

k221 is:  Flag Algebra Element over Rational Field
0    - Flag on 5 points, ftype from [] with edges=[]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3], [0, 1, 4]]
1/30 - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [0, 1, 3]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3]]
2/15 - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [0, 1, 3], [0, 2, 4]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] 

k311 is:  Flag Algebra Element over Rational Field
0    - Flag on 5 points, ftype from [] with edges=[]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3]]
0    

In [7]:
###
### This is the code that performs the calculations on the colored theory
###

#edge with correct colors
C = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]])
#pointed edge with correct colors
Cp = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], ftype=[0])

#edge with bad colors (since color-blind, this includes the bad edges looking the wrong way too)
B = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]])

#pointed edge with bad colors
Bp = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [2]], C1=[[1]], C2=[], ftype=[0])

#missing edge with good colors
M = CTGp(3, edges=[], C0=[[0]], C1=[[1]], C2=[[2]])

#positivity assumptions
#each point, good edges are more than bad edges (divided by two due to the wrong color order)
#edge density is larger than 0.194 (from previous calculation)
colored_assums = [Cp - Bp/2, C - 194/1000]

#an optimal construction
#optim = CTGp.blowup_construction(6, 3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], symmetric=True, symbolic=True)
#ssoptim = optim.set_sum()
#der_optims = ssoptim.derivatives([1/3, 1/3])

#bad is less than missing
res = optimize_problem_new(CTGp, B + (-1)*M, 6, maximize=True, positives=colored_assums)#, certificate=False, exact=False, construction=None)
#should be around 1.5177109841127238e-08
#print("\n\n", res[0], "\n\n")
#print("\n\n", res, "\n\n")

Base flags generated, their number is 2840
The relevant ftypes are constructed, their number is 20
Block sizes before symmetric/asymmetric change is applied: [35, 57, 82, 123, 123, 123, 40, 60, 60, 60, 60, 60, 20, 30, 30, 30, 30, 30, 30, 30]


Done with mult table for Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]], C0=[[0]], C1=[[1]], C2=[[2], [3]]: : 20it [00:02,  7.68it/s]    


Tables finished


Done with positivity constraint 1: 100%|██████████| 2/2 [00:01<00:00,  1.26it/s]


Constraints finished
Running sdp without construction. Used block sizes are [26, 9, 32, 25, 14, 68, 36, 87, 31, 92, 43, 80, 14, 26, 21, 39, 39, 21, 39, 21, 39, 21, 33, 27, 12, 8, 21, 9, 18, 12, 13, 17, 24, 6, 17, 13, 30, 14, 16, -2840, -45]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 2.33e-01 Pobj: -2.2590878e+01 Ad: 1.81e-01 Dobj:  4.2470228e+01 
Iter:  2 Ap: 9.09e-01 Pobj: -1.1659370e+02 Ad: 4.89e-01 Dobj:  5.0281198e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -1.4152973e+02 Ad: 7.70e-01 Dobj:  4.8104735e+00 
Iter:  4 Ap: 1.00e+00 Pobj: -1.5208608e+02 Ad: 8.96e-01 Dobj:  6.5482479e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -1.7241918e+02 Ad: 8.32e-01 Dobj:  1.0094385e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -2.0589389e+02 Ad: 6.17e-01 Dobj:  3.7667026e-02 
Iter:  7 Ap: 1.00e+00 Pobj: -2.0896256e+02 Ad: 5.45e-01 Dobj:  2.6921746e-02 
Iter:  8 Ap: 9.40e-01 Pobj: -1.6989471e+02 Ad: 6.85e-01 Dobj:  9.4731408e-03 
Iter:  9 Ap: 7.15e-01 Pobj: -1.7307377e+02 Ad:

In [4]:
def optimize_problem_new(self, target_element, target_size, maximize=True, positives=None, construction=None):
    from csdpy import solve_sdp
    from tqdm import tqdm
    import sys
    import io
    import time
    
    #
    # initial setup
    #
    
    if target_size not in self.sizes():
        raise ValueError("For theory {}, size {} is not allowed.".format(self._name, target_size))

    base_flags = self.generate_flags(target_size)
    print("Base flags generated, their number is {}".format(len(base_flags)))
    mult = -1 if maximize else 1
    target_vector_exact = (target_element.project()*(mult)<<(target_size - target_element.size())).values()

    #
    # create the table data
    #

    plausible_sizes = []
    for fs in self.sizes():
        if fs>=target_size:
            break
        if fs==0:
            continue
        plausible_sizes.append(fs)
    ftype_pairs = []
    for fs, ns in itertools.combinations(plausible_sizes, r=int(2)):
        if self.size_combine(fs, ns, ns) <= target_size:
            kk = ns-fs
            found = False
            for ii, (bfs, bns) in enumerate(ftype_pairs):
                if bns-bfs==kk:
                    found = True
                    if ns>bns:
                        ftype_pairs[ii]=(fs, ns)
                    break
            if not found:
                ftype_pairs.append((fs, ns))

    ftype_data = []
    for fs, ns in ftype_pairs:
        ftype_flags = self.generate_flags(fs)
        ftypes = [flag.subflag([], ftype_points=list(range(fs))) for flag in ftype_flags]
        for xx in ftypes:
            ftype_data.append((ns, xx, target_size))
    ftype_data.sort()

    print("The relevant ftypes are constructed, their number is {}".format(len(ftype_data)))

    flags = [self.generate_flags(dat[0], dat[1]) for dat in ftype_data]
    flag_sizes = [len(xx) for xx in flags]

    print("Block sizes before symmetric/asymmetric change is applied: {}".format(flag_sizes))

    sym_asym_mats = [self.sym_asym_bases(dat[0], dat[1]) for dat in ftype_data]

    table_constructor = {}
    for ii, dat in (pbar := tqdm(enumerate(ftype_data))):
        ns, ftype, target_size = dat
        #pre-calculate the table here
        table = self.mul_project_table(ns, ns, ftype, ftype_inj=[], target_size=target_size)
        if table==None:
            print("Structures with size {} and {} had singular multiplication table!".format(ns, ftype))
            continue
        sym_base, asym_base = sym_asym_mats[ii]
        bases = []
        if sym_base.nrows()!=0:
            bases.append(sym_base)
        if asym_base.nrows()!=0:
            bases.append(asym_base)
        save_stuff("colored table {} new".format(ii), table)
        save_stuff("colored sym asym {} new".format(ii), (sym_base, asym_base))
        table_constructor[dat] = bases
        pbar.set_description("Done with mult table for {}".format(ftype))

    sdp_data = self._tables_to_sdp_data(table_constructor)
    print("Tables finished", flush=True)
    
    #
    # add constraints data
    #
    
    if positives == None:
        positives_list_exact = []
        constraints_vals = []
    else:
        positives_list_exact = []
        for ii in (pbar:= tqdm(range(len(positives)))):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            kf = fv.ftype().size()
            nf = fv.size()
            if self._size_combine == None:
                df = target_size - nf + kf
            else:
                df = -1
                for xx in self.sizes():
                    if self._size_combine(kf, nf, xx)==target_size:
                        df = xx
                        break
            mult_table = self.mul_project_table(nf, df, fv.ftype(), ftype_inj=[], target_size=target_size)
            fvvals = fv.values()
            m = matrix(QQ, [vector(fvvals*mat) for mat in mult_table])
            positives_list_exact += list(m.T)
            pbar.set_description("Done with positivity constraint {}".format(ii))
        constraints_vals = [0]*len(positives_list_exact)
    if target_element.ftype().size()==0:
        one_vector = vector([1]*len(base_flags))
    else:
        one_vector = (target_element.ftype().project()<<(target_size - target_element.ftype().size())).values()
    positives_list_exact.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    positives_matrix_exact = matrix(positives_list_exact)
    save_stuff("colored positives new", positives_matrix_exact)
    sdp_data = self._constraints_to_sdp_data(len(base_flags), constraints_vals, positives_list_exact, sdp_data)
    print("Constraints finished")

    save_stuff("colored sdp data new", sdp_data)
    #
    # if no y value provided, run the optimizer first, only to get the y values
    #
    if construction==None:
        mat_inds, mat_vals, block_sizes = sdp_data
        print("Running sdp without construction. Used block sizes are {}".format(block_sizes))


        time.sleep(float(0.1))
        initial_sol = solve_sdp(block_sizes, list(target_vector_exact), mat_inds, mat_vals)
        time.sleep(float(0.1))

        res = initial_sol['primal'] * (-1 if maximize else 1)
        return res
    else:
        if isinstance(construction, FlagAlgebraElement):
            phi_vectors_exact = [construction.values()]
        else:
            phi_vectors_exact = [xx.values() for xx in construction]

    #
    # adjust the table to consider the kernel from y_rounded
    #

    print("Adjusting table with kernels from construction")
    table_constructor = self._adjust_table_phi(table_constructor, phi_vectors_exact)

    sdp_data = self._tables_to_sdp_data(table_constructor)
    sdp_data = self._constraints_to_sdp_data(len(base_flags), constraints_vals, positives_list_exact, sdp_data)
    mat_inds, mat_vals, block_sizes = sdp_data

    print("Running SDP after kernel correction. Used block sizes are {}".format(block_sizes))

    time.sleep(float(0.1))
    final_sdp = solve_sdp(block_sizes, list(target_vector_exact), mat_inds, mat_vals)
    time.sleep(float(0.1))

    res = final_sdp['primal'] * (-1 if maximize else 1)
    return res 

In [20]:
def optimize_problem_old(self, target_element, target_size, \
                         ftypes=None, maximize=True, certificate=False, \
                         positives=None):
    import time
    
    from csdpy import solve_sdp
    import numpy as np
    from tqdm import tqdm
    import sys
    
    current = time.time()
    #calculate constraints from positive vectors
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
        constraints_vals = [0]*len(constraints_flags)
    
    #calculate ftypes
    if ftypes is None:
        flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
                  for flag in self.generate_flags(kk)]
        ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
                  for flag in flags]

    print("Ftypes constructed in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    block_sizes = [len(self.generate_flags((target_size + \
                   ftype.size())//2, ftype)) for ftype in ftypes]
    constraints = len(self.generate_flags(target_size))
    
    alg = FlagAlgebra(QQ, self)
    
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])

    block_sizes.extend([-constraints, -2*len(constraints_vals)])
    block_num = len(block_sizes)
    mat_inds = []
    mat_vals = []
    print("Block sizes done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    print("Block sizes are {}".format(block_sizes), flush=True)
    print("Calculating product matrices for {} ftypes and {} structures".format(len(ftypes), constraints), flush=True)
    for ii, ftype in (pbar := tqdm(enumerate(ftypes), file=sys.stdout)):
        ns = (target_size + ftype.size())//2
        fls = self.generate_flags(ns, ftype)
        table = self.mul_project_table(ns, ns, ftype, [])
        save_stuff("color table {} old".format(ii), table)
        for gg, mm in enumerate(table):
            dd = mm._dict()
            if len(dd)>0:
                inds, values = zip(*mm._dict().items())
                iinds, jinds = zip(*inds)
                for cc in range(len(iinds)):
                    if iinds[cc]>=jinds[cc]:
                        mat_inds.extend([gg+1, ii+1, iinds[cc]+1, 
                                         jinds[cc]+1])
                        mat_vals.append(values[cc])
        pbar.set_description("{} is complete".format(ftype))
    
    print("Table calculation done in {:.2f}s".format(time.time() - current), flush=True); current = time.time()
    if maximize:
        avals = (target_element.project()*(-1)<<(target_size - \
                                       target_element.size())).values()
    else:
        avals = (target_element.project()<<(target_size - \
                                  target_element.size())).values()

    for ii in range(len(constraints_vals)):
        mat_inds.extend([0, block_num, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    constraints_flags_vec = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    save_stuff("color positives old", matrix(constraints_flags_vec))
    for gg in range(constraints):
        mat_inds.extend([gg+1, block_num-1, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(len(constraints_flags_vec)):
            mat_inds.extend([gg+1, block_num, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    print("Target and constraint calculation done in {:.2f}s\n".format(time.time() - current), flush=True); current = time.time()
    save_stuff("color sdp data old", (mat_inds, mat_vals, block_sizes))
    sdp_result = solve_sdp(block_sizes, list(avals), 
                           mat_inds, mat_vals)
    if maximize:
        ret = max(-sdp_result['primal'], -sdp_result['dual'])
    else:
        ret = min(sdp_result['primal'], sdp_result['dual'])
    print("Result is {}".format(ret), flush=True)
    if certificate:
        ralg = FlagAlgebra(RR, self)
        vec = ralg(target_size, sdp_result['y'])
        ret = (ret, vec, sdp_result)
    return ret

In [6]:
def save_stuff(save_name, data):
    with open(save_name, 'wb') as file:
        pickle.dump(data, file)

In [22]:
t_old = load_stuff("color table {} old".format(0))
t_new = load_stuff("colored table {} new".format(0))
print(t_old[1]._dict(), t_new[1]._dict())

{(0, 1): 1/3, (1, 0): 1/3} {(0, 1): 1/3, (1, 0): 1/3}


In [30]:
asd = load_stuff("color positives old")
dsa = load_stuff("colored positives new")
for ii in range(45):
    if list(asd)[ii]!=list(dsa)[ii]:
        print(ii)

42
43
44


In [24]:
from tqdm import tqdm
def load_stuff(name):
    with open(name, "rb") as file:
        ret = pickle.load(file)
    return ret

for ii in range(20):
    t_old = load_stuff("color table {} old".format(ii))
    t_new = load_stuff("colored table {} new".format(ii))
    if len(t_old) != len(t_new):
        print("{}: has {} vs {}".format(ii, len(t_old), len(t_new)))
    else:
        for jj in tqdm(range(len(t_old))):
            if t_old[jj]!=t_new[jj]:
                print("{}: matrix {} is diff".format(ii, jj))

100%|███████████████████████████████████| 2840/2840 [00:00<00:00, 437822.01it/s]
